In [1]:
import json, requests, threading
import getpass
from bs4 import BeautifulSoup
from Requester import Requester

### Get Password

In [2]:
password = getpass.getpass(prompt="Enter Password Plis: ")

Enter Password Plis: ········


### Create Session

In [3]:
session = requests.Session()
response = session.post("https://ayamel.byu.edu/auth/password", {"username":"admin", "password":password}, allow_redirects=False)
response = session.get("https://ayamel.byu.edu/admin/content")
session.close()

[('PLAY_FLASH', 'success=Welcome+Admin%21'), ('PLAY_SESSION', '12c5dd4be3a2dfb1431f8f9835ae540162281866-userId=13')]


### Extract Data from Content (html)

In [4]:
class Content:
    def __init__(self, resourceId, name, owner, lang, content_type):
        self.resourceId = resourceId
        self.name = name
        self.owner = owner
        self.lang = lang
        self.content_type = content_type

def extract(row):
    fields = row.find_all('td')
    rid = fields[6].text
    name = fields[1].text
    owner = fields[2].text
    lang = fields[3].text
    content_type = fields[7].text
    return Content(rid, name, owner, lang, content_type)

In [5]:
soup = BeautifulSoup(response.text, "html5lib")
rows = soup.find_all('tr')[1:] # get rid of the first row cause some idiots didn't use table headers
content_list = [extract(x) for x in rows]

### Get Resources

In [6]:
from Requester import CONTENT_OBJS
from Requester import RESOURCE_OBJS

In [7]:
NUM_THREADS = 20

In [8]:
def run_request_process():
    pool = []
    for i in range(NUM_THREADS): pool.append(Requester())
    for t in pool: t.start()
    for t in pool: t.join()

In [9]:
CONTENT_OBJS.extend(content_list) #wedontcare

In [10]:
run_request_process()

### Graphing

In [13]:
import pandas
import numpy as np

In [70]:
# mapping function, convert json to list of values
def values(obj):
    uri = ""
    if obj["status"] == "awaiting_content":
        print("Awaiting content: %s" % str(obj))
        return ["", "", "", "", "", "", ""]
    
    files = obj["content"]["files"][0]
    if "streamUri" in files.keys():
        uri = files["streamUri"]
        
    return [obj["id"], obj["title"], obj["type"], obj["dateAdded"],
            obj["dateModified"], uri, obj["content"]["files"][0]["mimeType"]]

headers = ["id", "title", "type", "dateAdded", "dateModified", "streamUri", "mimeType"]
pandas_rows = [ values(o) for o in RESOURCE_OBJS]

Awaiting content: {'relations': [], 'title': 'Test Video3', 'dateAdded': '1443643111', 'client': {'id': 'byu_demo', 'name': 'BYU Demos'}, 'id': '560c3ee733e57c500b8b4569', 'status': 'awaiting_content', 'dateModified': '1443643111', 'type': 'video'}


In [71]:
data = np.array(pandas_rows)

In [72]:
all_data = pandas.DataFrame(data=data, columns=headers)

In [77]:
def getRowsContainingString(tupl):
    return all_data.loc[all_data[tupl[0]].str.contains(tupl[1])]
    
getRowsContainingString(("mimeType", "brightcove"))

,id,title,type,dateAdded,dateModified,streamUri,mimeType
0,53064e3b1bc7cb0951000001,Les enfants du marais,video,1392922171,1440011241,brightcove://4316245445001,video/x-brightcove
1,5314cbf41bc7cbdf70000000,Die fetten Jahre sind vorbei,video,1393871860,1435616555,brightcove://4316074209001,video/x-brightcove
2,535839111bc7cb1572000000,Die drei Räuber,video,1398290705,1435690758,brightcove://4315901956001,video/x-brightcove
5,54c3df1f33e57c52758b4567,Lola Rennt,video,1422122783,1435617250,brightcove://4007160461001,video/x-brightcove
6,54d64cd733e57c613e8b4567,Sophie Scholl - die letzten Tage,video,1423330519,1456435013,brightcove://4036669831001,video/x-brightcove
12,5663974d33e57c99678b456d,06-Estados Unidos,video,1449367373,1504813141,brightcove://4647694950001,video/x-brightcove
13,5663974d33e57c99678b4571,01-México,video,1449367373,1449367375,brightcove://4647747897001,video/x-brightcove
14,5663974e33e57c99678b4575,36-España,video,1449367374,1449367375,brightcove://4647694936001,video/x-brightcove
15,5663975033e57c9c678b456d,04-Colombia,video,1449367376,1463431036,brightcove://4647772887001,video/x-brightcove
16,5663975133e57c99678b457d,02-Bolivia,video,1449367377,1449367378,brightcove://4647747854001,video/x-brightcove
